<a href="https://colab.research.google.com/github/yinma-ml/machine-learning/blob/master/Gender_detection_DeepFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Gender dectection based on human speech

**DeepFM**

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import log_loss, roc_auc_score
!pip install deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, get_feature_names


     |████████████████████████████████| 112kB 3.4MB/s 


In [5]:
data = pd.read_csv('/voice.csv')  #3168 rows × 21 columns

In [6]:
print(data.head())
print(data.isnull().sum())  #no null value

   meanfreq        sd    median       Q25  ...    maxdom   dfrange   modindx  label
0  0.059781  0.064241  0.032027  0.015071  ...  0.007812  0.000000  0.000000   male
1  0.066009  0.067310  0.040229  0.019414  ...  0.054688  0.046875  0.052632   male
2  0.077316  0.083829  0.036718  0.008701  ...  0.015625  0.007812  0.046512   male
3  0.151228  0.072111  0.158011  0.096582  ...  0.562500  0.554688  0.247119   male
4  0.135120  0.079146  0.124656  0.078720  ...  5.484375  5.476562  0.208274   male

[5 rows x 21 columns]
meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64


In [7]:
print('sample:{}'.format(data.shape[0]))
print('male：{}'.format(data[data.label=='male'].shape[0]))
print('female：{}'.format(data[data.label=='female'].shape[0]))

sample:3168
male：1584
female：1584


In [9]:
X = data.iloc[:,:-1]  #everything except the last column
sparse_features = [i for i in X.columns]
target = data.iloc[:,-1]
target_label = ['label']
print(X.head)
print(target.head)

<bound method NDFrame.head of       meanfreq        sd    median  ...    maxdom   dfrange   modindx
0     0.059781  0.064241  0.032027  ...  0.007812  0.000000  0.000000
1     0.066009  0.067310  0.040229  ...  0.054688  0.046875  0.052632
2     0.077316  0.083829  0.036718  ...  0.015625  0.007812  0.046512
3     0.151228  0.072111  0.158011  ...  0.562500  0.554688  0.247119
4     0.135120  0.079146  0.124656  ...  5.484375  5.476562  0.208274
...        ...       ...       ...  ...       ...       ...       ...
3163  0.131884  0.084734  0.153707  ...  4.210938  4.203125  0.161929
3164  0.116221  0.089221  0.076758  ...  3.679688  3.640625  0.277897
3165  0.142056  0.095798  0.183731  ...  2.937500  2.929688  0.194759
3166  0.143659  0.090628  0.184976  ...  3.593750  3.585938  0.311002
3167  0.165509  0.092884  0.183044  ...  0.554688  0.546875  0.350000

[3168 rows x 20 columns]>
<bound method NDFrame.head of 0         male
1         male
2         male
3         male
4         mal

In [10]:
# 
lbe = LabelEncoder()
data['label'] = lbe.fit_transform(target)  # male female to 1 0 


In [11]:
# normalization
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[sparse_features] = mms.fit_transform(data[sparse_features])

In [12]:
# count #unique features for each sparse field, and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features]
# print(fixlen_feature_columns)

linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
# print('feature name:',feature_names)


In [16]:
# split train test sets, test size=0.2
trainset, testset = train_test_split(data, test_size=0.2)

train_model_input = {name:trainset[name].values for name in feature_names}
# print('train_model_input:', train_model_input)
test_model_input = {name:testset[name].values for name in feature_names}
print('test_model_input:', test_model_input)

test_model_input: {'meanfreq': array([0.96176935, 0.1699842 , 0.84107425, 0.38988942, 0.2385466 ,
       0.7042654 , 0.55860979, 0.85845182, 0.2628752 , 0.48151659,
       0.07109005, 0.52827804, 0.63317536, 0.74186414, 0.50110585,
       0.08530806, 0.8821485 , 0.68973144, 0.49541864, 0.45023697,
       0.64391785, 0.50205371, 0.73459716, 0.86761453, 0.89668246,
       0.37661927, 0.8300158 , 0.81453397, 0.24518167, 0.57472354,
       0.46508689, 0.84012638, 0.84960506, 0.6436019 , 0.87993681,
       0.67393365, 0.91753555, 0.82085308, 0.80157978, 0.3699842 ,
       0.08183254, 0.65655608, 0.41832543, 0.45339652, 0.41674566,
       0.85023697, 0.29510269, 0.82306477, 0.6906793 , 0.0549763 ,
       0.11974724, 0.04518167, 0.37219589, 0.37187994, 0.95545024,
       0.89478673, 0.39778831, 0.21895735, 0.92511848, 0.56334913,
       0.02812006, 0.54660348, 0.19462875, 0.54028436, 0.19146919,
       0.78104265, 0.65781991, 0.02432859, 0.28562401, 0.71153239,
       0.09447077, 0.42622433, 

In [14]:
##DeepFM

In [18]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
# Loss, mse ,  logic regression :binary_crossentropy
model.compile('adam', 'binary_crossentropy', metrics=['binary_crossentropy'], )

# 
history = model.fit(train_model_input, trainset[target_label].values, batch_size=256, epochs=10,verbose=2, validation_split=0.2)
                    # verbose=2, )
                    # validation_split=0.2, )

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


8/8 - 1s - loss: 0.6939 - binary_crossentropy: 0.6939 - val_loss: 0.6966 - val_binary_crossentropy: 0.6966
Epoch 2/10
8/8 - 0s - loss: 0.6932 - binary_crossentropy: 0.6932 - val_loss: 0.6935 - val_binary_crossentropy: 0.6935
Epoch 3/10
8/8 - 0s - loss: 0.6931 - binary_crossentropy: 0.6931 - val_loss: 0.6940 - val_binary_crossentropy: 0.6940
Epoch 4/10
8/8 - 0s - loss: 0.6930 - binary_crossentropy: 0.6930 - val_loss: 0.6940 - val_binary_crossentropy: 0.6940
Epoch 5/10
8/8 - 0s - loss: 0.6931 - binary_crossentropy: 0.6931 - val_loss: 0.6947 - val_binary_crossentropy: 0.6947
Epoch 6/10
8/8 - 0s - loss: 0.6930 - binary_crossentropy: 0.6930 - val_loss: 0.6947 - val_binary_crossentropy: 0.6947
Epoch 7/10
8/8 - 0s - loss: 0.6933 - binary_crossentropy: 0.6933 - val_loss: 0.6938 - val_binary_crossentropy: 0.6938
Epoch 8/10
8/8 - 0s - loss: 0.6930 - binary_crossentropy: 0.6930 - val_loss: 0.6953 - val_binary_crossentropy: 0.6953
Epoch 9/10
8/8 - 0s - loss: 0.6930 - binary_crossentropy: 0.6930 - 

In [19]:
pred_ans = model.predict(test_model_input, batch_size=256)
print('pred_ans',pred_ans)

pred_ans [[0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.49207437]
 [0.4920743

In [20]:
print('test LogLoss', round(log_loss(testset[target_label].values, pred_ans), 4))
print('test AUC', round(roc_auc_score(testset[target_label].values, pred_ans), 4))

test LogLoss 0.693
test AUC 0.5032
